In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K


import pandas as pd
import numpy as np


In [8]:
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


In [10]:
#Load a previously saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/mai/ep_937_mse_0.05_r2_0.99.hdf5',
                                   custom_objects={'r_squared': r_squared},
                                   compile=True)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 44)]              0         
_________________________________________________________________
dense_20 (Dense)             (None, 1056)              47520     
_________________________________________________________________
dropout_16 (Dropout)         (None, 1056)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 1056)              1116192   
_________________________________________________________________
dropout_17 (Dropout)         (None, 1056)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 1056)              1116192   
_________________________________________________________________
dropout_18 (Dropout)         (None, 1056)              0   

In [17]:
ch_names = np.arange(44).tolist()
freqs = np.arange(191.6, 195.9, 0.1).tolist()
for x in range(len(ch_names)):
  ch_names[x] = "Ch %.0f (%.1f)" % (ch_names[x], freqs[x])

In [98]:
#set the channel number here
channel = 43

In [99]:
#Using a dataset with 1 ch constantly on for the test set
#retreiving both datasets from GitHub



git_xval = ('https://raw.githubusercontent.com/p-owens/MAI/main/data%20gen/ch{0}/x_val.csv'.format(channel))
git_yval = ('https://raw.githubusercontent.com/p-owens/MAI/main/data%20gen/ch{0}/y_val.csv'.format(channel))
dfx = pd.read_csv(git_xval,
                  index_col=False,
                  header=None,
                  names=ch_names                                   
                 )
dfy = pd.read_csv(git_yval,
                  index_col=False,
                  header=None,
                  names=ch_names
                 )
x_test = dfx.to_numpy()
y_test = dfy.to_numpy()

#removing the frequency row - treating these values as labels
x_test = x_test[1::2,:]

In [100]:
#cell to filter out the training samples where the chosen 
#channel is not negative
##########################

dfx = dfx.iloc[1::2]
df_chx = dfx[dfx[ch_names[channel]] !=0].iloc[:,:]
df_chy = dfy[dfy[ch_names[channel]] !=0].iloc[:,:]
#df_chy = df_chy[ch_names[channel]]


x_test = df_chx.to_numpy()
y_test = df_chy.to_numpy()

In [103]:
x_test

array([], shape=(0, 44), dtype=float64)

In [101]:
#calculate the channel specific mean squared error
#this method included the errors for 0 values 
#(if those test samples haven't been filtered out)
#it calculates the mean absolute error and  
#then divides my the mean channel output value
##########################

mape = tf.keras.metrics.MeanAbsolutePercentageError()

mape.update_state([y_test[:,channel]], [model.predict(x_test)[:,channel]]).numpy()
mape.result().numpy() / np.mean(y_test[:,channel])

ValueError: ignored

In [102]:
#evaluate the model on the test set
results = model.evaluate(x_test,y_test, verbose=1)
print("{name1}\t\t: {mse:<.4f}\t(lower is better)\n{name2}\t: {r2:<.2%}\t(higher is better)".format(name1 = model.metrics_names[0], name2 = model.metrics_names[1],mse = results[0],r2 = results[1]))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:583: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: ignored

In [21]:
print(np.mean(pd.DataFrame(y_test[:,channel])))

0    0.864838
dtype: float64


In [22]:
# cant use percentage error becaue we have some samples where the true value is 0
# instead use relative percentage difference 
# see https://en.wikipedia.org/wiki/Relative_change_and_difference

def rpd(x, y):
  epsilon = 1e-20
  return abs(x - y) / ((abs(x) + abs(y) + epsilon) / 2) * 50

In [23]:
pd.DataFrame([y_test[:,channel],
model.predict(x_test)[:,channel], 
rpd(model.predict(x_test)[:,channel], y_test[:,channel])])#, np.transpose(sq_err))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
0,0.000000,0.000000,0.0,1.629071,1.726116,1.662034,0.0,1.383885,1.739002,1.854044,1.723892,1.721789,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.606497,1.735242,0.000000,1.615511,1.688885,1.648390,1.668745,0.000000,0.0,1.639117,1.736561,0.0,0.000000,1.670475,0.0,1.971242,0.000000,0.0,0.000000,0.0,0.0,1.785747,...,1.425903,1.628621,1.574962,1.693675,1.673863,1.577648,0.0,1.582159,0.0,1.625698,1.613318,0.0,0.0,0.0,0.0,0.000000,1.578774,1.583507,1.672361,1.668745,2.014374,0.0,1.800073,1.634105,1.430342,0.0,0.000000,1.612093,0.0,0.0,1.460932,0.000000,1.884256,0.000000,0.000000,1.668092,1.587998,0.000000,1.965079,0.0
1,0.112933,0.009264,0.0,1.506751,1.532381,1.599063,0.0,0.870434,1.445919,1.412158,1.381154,1.544537,0.058001,0.0,0.426918,0.131407,0.037181,0.122847,1.293617,1.639776,0.156569,1.364848,1.546086,1.548160,1.640279,0.612153,0.0,1.532112,1.439010,0.0,0.211744,1.605742,0.0,1.522483,0.291966,0.0,0.000337,0.0,0.0,1.386163,...,1.023224,1.202144,1.319638,1.543579,1.489366,1.308736,0.0,1.432709,0.0,1.416934,1.081258,0.0,0.0,0.0,0.0,0.273647,1.257554,1.419380,1.255990,1.640279,1.613980,0.0,1.627739,1.468915,1.105796,0.0,0.299644,1.417241,0.0,0.0,1.143669,0.284315,1.624873,0.228798,0.026022,1.367749,1.463957,0.598232,1.924231,0.0
2,100.000000,100.000000,0.0,3.900740,5.945541,1.930986,0.0,22.776331,9.202200,13.529026,11.038098,5.426657,100.000000,0.0,100.000000,100.000000,100.000000,100.000000,10.788537,2.828609,100.000000,8.410523,4.414221,3.135565,0.860255,100.000000,0.0,3.374261,9.369998,0.0,100.000000,1.975834,0.0,12.844729,100.000000,0.0,100.000000,0.0,0.0,12.597587,...,16.441761,15.065801,8.820678,4.636506,5.832573,9.316568,0.0,4.957081,0.0,6.861306,19.745603,0.0,0.0,0.0,0.0,100.000000,11.325208,5.465635,14.218621,0.860255,11.035131,0.0,5.027506,5.323513,12.796851,0.0,100.000000,6.432176,0.0,0.0,12.180888,100.000000,7.391673,100.000000,100.000000,9.893227,4.064304,100.000000,1.050251,0.0


In [24]:
np.mean(rpd(model.predict(x_test)[:,channel], y_test[:,channel]))

23.663028621174945

In [45]:
m = tf.keras.metrics.MeanAbsolutePercentageError()
m.update_state([[0.1, 0.2]], [[0.2, 0.5]])
m.result().numpy()


125.0

In [86]:
channel = [0,3,5,7,11]

In [88]:
for i in range(len(channel)):
  print(channel[i])

0
3
5
7
11


In [105]:
#set the channel number here
channel = [0,3,5,7,9,11, 13, 15 ,17, 19, 21, 23, 25, 27, 28, 30, 31, 32, 33, 35, 37, 38, 39, 40, 41 ]
pct_err = []


#Using a dataset with 1 ch constantly on for the test set
#retreiving both datasets from GitHub


for i in range(len(channel)):
  print(channel[i])
  git_xval = ('https://raw.githubusercontent.com/p-owens/MAI/main/data%20gen/ch{0}/x_val.csv'.format(channel[i]))
  git_yval = ('https://raw.githubusercontent.com/p-owens/MAI/main/data%20gen/ch{0}/y_val.csv'.format(channel[i]))
  dfx = pd.read_csv(git_xval,
                    index_col=False,
                    header=None,
                    names=ch_names                                   
                  )
  dfy = pd.read_csv(git_yval,
                    index_col=False,
                    header=None,
                    names=ch_names
                  )
  x_test = dfx.to_numpy()
  y_test = dfy.to_numpy()

  #removing the frequency row - treating these values as labels
  x_test = x_test[1::2,:]
  #cell to filter out the training samples where the chosen 
  #channel[i] is not negative
  ##########################

  dfx = dfx.iloc[1::2]
  df_chx = dfx[dfx[ch_names[channel[i]]] !=0].iloc[:,:]
  df_chy = dfy[dfy[ch_names[channel[i]]] !=0].iloc[:,:]
  #df_chy = df_chy[ch_names[channel[i]]]


  x_test = df_chx.to_numpy()
  y_test = df_chy.to_numpy()
  #calculate the channel specific mean squared error
  #this method included the errors for 0 values 
  #(if those test samples haven't been filtered out)
  #it calculates the mean absolute error and  
  #then divides my the mean channel output value
  ##########################

  mape = tf.keras.metrics.MeanAbsolutePercentageError()

  mape.update_state([y_test[:,channel[i]]], [model.predict(x_test)[:,channel[i]]]).numpy()
  pct_err.append(mape.result().numpy() / np.mean(y_test[:,channel[i]]))

0
3
5
7


KeyboardInterrupt: ignored

In [97]:
pct_err

[12.318003750631462,
 8.20983630400927,
 6.089566568660994,
 4.469217648364095,
 4.443249706433819,
 5.055869292460759,
 5.776496729032451,
 7.044206464627855,
 7.078317331027594,
 6.109780041989137,
 4.756566470671473,
 3.4578734920098233,
 3.2304070974396417,
 3.391581704292962,
 3.250158944712763,
 4.11210728163074,
 4.17296818624338,
 4.6229131528494225,
 4.15428849561241,
 2.2698434182824254,
 0.8890991300576628,
 0.6349524488524456,
 0.5704269782634673,
 0.3297490271901366,
 0.22345757722467283]

In [107]:
channel

[0,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 28,
 30,
 31,
 32,
 33,
 35,
 37,
 38,
 39,
 40,
 41]